<a href="https://colab.research.google.com/github/jacobpad/jacobpad.github.io/blob/master/data/U1_Build_Week_Jacob_Padgett.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# A bunch of imports, yes, it's probably overkill
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
import seaborn as sns
import scipy.stats as stats
import math
from sklearn.decomposition import PCA # You don't necessarily have to use this
from sklearn.cluster import KMeans # You don't necessarily have to use this
from sklearn.preprocessing import StandardScaler # You don't necessarily have to use this
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO

# Import & clean/reduce data

In [2]:
# URL
url = 'https://raw.githubusercontent.com/jacobpad/jacobpad.github.io/master/data/all_month.csv'

# Read in dataset
df = pd.read_csv(url)
df.head(2)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-01-28T23:20:04.732Z,61.427500,-149.908900,30.80,1.60,ml,NaN,NaN,NaN,0.63,ak,ak0201aouzfc,2020-01-28T23:25:38.638Z,"10km SSE of Big Lake, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
1,2020-01-28T23:18:37.420Z,35.057333,-118.322167,0.07,1.99,ml,34.0,34.0,0.06846,0.23,ci,ci39285400,2020-01-28T23:22:44.432Z,"14km W of Mojave, CA",earthquake,0.29,31.61,0.227,26.0,automatic,ci,ci


In [3]:
# Drop the columns I don't want - time, id & updated
df = df.drop(columns=['id', 'updated', 'horizontalError', 'depthError', 
                      'gap', 'magError', 'magNst', 'dmin', 'nst',
                      'locationSource',	'magSource'])
df.head(2)

,time,latitude,longitude,depth,mag,magType,rms,net,place,type,status
0,2020-01-28T23:20:04.732Z,61.427500,-149.908900,30.80,1.60,ml,0.63,ak,"10km SSE of Big Lake, Alaska",earthquake,automatic
1,2020-01-28T23:18:37.420Z,35.057333,-118.322167,0.07,1.99,ml,0.23,ci,"14km W of Mojave, CA",earthquake,automatic


In [4]:
import pandas_profiling
profile = pandas_profiling.ProfileReport(df)
profile

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,11
Number of observations,12397
Total Missing (%),0.0%
Total size in memory,1.0 MiB
Average record size in memory,88.0 B
Numeric,5
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [5]:
# See what the type of each column is
df.dtypes

time          object
latitude     float64
longitude    float64
depth        float64
mag          float64
magType       object
rms          float64
net           object
place         object
type          object
status        object
dtype: object

In [6]:
# How many nulls are there?
df.isnull().sum().sort_values()

time         0
latitude     0
longitude    0
depth        0
rms          0
net          0
place        0
type         0
status       0
mag          1
magType      1
dtype: int64

In [7]:
# See all rows with null values
df[pd.isnull(df).any(axis=1)]

,time,latitude,longitude,depth,mag,magType,rms,net,place,type,status
3363,2020-01-19T05:34:29.270Z,38.773667,-122.7175,2.28,NaN,NaN,0.02,nc,"2km W of Anderson Springs, CA",earthquake,reviewed


In [8]:
df['magType'].isnull().value_counts()

False    12396
True         1
Name: magType, dtype: int64

In [9]:
# See the magType value counts
df['magType'].value_counts() #.sum()

ml       7464
md       4151
mb        643
mww        88
mwr        26
mb_lg      10
mh          5
mwp         5
mw          4
Name: magType, dtype: int64

In [0]:
# filter based on specific conditions
# df_new = df[df['mag'] == 2.28]
# df_new

In [11]:
df.isnull().sum().sort_values()

time         0
latitude     0
longitude    0
depth        0
rms          0
net          0
place        0
type         0
status       0
mag          1
magType      1
dtype: int64

In [12]:
# See the index of the mising values in my mag & magType columns
print(df.shape)
df[3363:3364]

(12397, 11)


,time,latitude,longitude,depth,mag,magType,rms,net,place,type,status
3363,2020-01-19T05:34:29.270Z,38.773667,-122.7175,2.28,NaN,NaN,0.02,nc,"2km W of Anderson Springs, CA",earthquake,reviewed


In [0]:
df.drop(index=3363, inplace=True)

In [14]:
# Now that it's gone, reset the index
df.reset_index(inplace=True)
df.shape

(12396, 12)

In [15]:
# See, it's reset
df[3362:3365]

,index,time,latitude,longitude,depth,mag,magType,rms,net,place,type,status
3362,3362,2020-01-19T05:37:32.960Z,33.996167,-117.178000,15.15,0.58,ml,0.14,ci,"7km S of Redlands, CA",earthquake,reviewed
3363,3364,2020-01-19T05:27:21.130Z,35.622667,-117.440167,9.60,0.90,ml,0.17,ci,"16km SSW of Searles Valley, CA",earthquake,reviewed
3364,3365,2020-01-19T05:24:16.990Z,17.860000,-66.873600,12.00,2.80,md,0.12,pr,"12km SSE of Guanica, Puerto Rico",earthquake,reviewed


In [16]:
# What's still null?
df.isnull().sum().sort_values()

index        0
time         0
latitude     0
longitude    0
depth        0
mag          0
magType      0
rms          0
net          0
place        0
type         0
status       0
dtype: int64

# Begin some graphs

In [0]:
# This takes a long time to pull up
sns.pairplot(df)
plt.show();

In [18]:
# Remind me of dtypes
df.dtypes

index          int64
time          object
latitude     float64
longitude    float64
depth        float64
mag          float64
magType       object
rms          float64
net           object
place         object
type          object
status        object
dtype: object

In [0]:
time_df    = pd.Series(df['time'])
lat_df     = pd.Series(df['latitude'])
lon_df     = pd.Series(df['longitude'])
depth_df   = pd.Series(df['depth'])
mag_df     = pd.Series(df['mag'])
magType_df = pd.Series(df['magType'])
rms_df     = pd.Series(df['rms'])
net_df     = pd.Series(df['net'])
place_df   = pd.Series(df['place'])
type_df    = pd.Series(df['type'])
status_df  = pd.Series(df['status'])

In [0]:
# Plot the magnitude and quantity of quakes
plt.hist(mag_df)

plt.title('Earthquakes in the Last 30* Days')
plt.text(x=-2,
         y=-1000,
         s='* Record taken from Dec 29, 2019 - Jan 28, 2020')
plt.xlabel('Magnitude')
plt.ylabel('Number of Earthquakes')
plt.show()

In [0]:
sns.distplot(mag_df);

In [22]:
# Why does it appear dome of the magnitudes are negative
mag_df.unique()

array([ 1.6 ,  1.99,  1.66,  0.86,  4.4 ,  1.9 ,  4.7 ,  2.11,  0.75,
        0.6 ,  1.37,  0.89,  6.1 ,  4.5 ,  1.8 ,  1.58,  1.06,  1.65,
        2.07,  0.58,  1.43,  4.9 ,  2.3 ,  0.57,  5.  ,  4.1 ,  5.1 ,
        2.09,  2.5 ,  1.47,  3.9 ,  1.67,  7.7 ,  1.23,  1.77,  1.93,
        1.38,  1.08,  1.44,  0.2 ,  1.1 ,  5.5 ,  0.71,  0.98,  3.08,
        0.3 ,  0.88,  1.5 ,  1.01,  0.99,  0.61,  1.48,  0.56,  5.3 ,
        2.19,  1.59,  3.13,  1.04,  1.79,  0.7 ,  4.3 ,  1.71,  2.4 ,
        1.52,  5.4 ,  1.74,  0.37,  3.4 ,  1.92,  3.6 ,  2.52,  3.15,
        2.  ,  0.07,  1.4 ,  1.81,  1.82,  4.8 ,  1.54,  1.96,  1.75,
        1.2 ,  0.13,  3.32,  0.72,  2.15,  0.8 ,  0.67,  1.18,  1.3 ,
        0.44,  1.53,  3.61,  0.84,  1.03,  1.26,  1.34,  2.21,  1.83,
        1.45,  0.91,  0.95, -0.02,  0.49,  0.36,  2.1 ,  2.14,  0.62,
        1.05,  0.25,  0.17,  1.07,  0.39,  0.  ,  2.35,  0.76,  2.37,
        1.46,  1.97,  2.33,  0.4 ,  0.27,  0.38,  1.95,  2.01,  2.26,
        0.74,  1.51,

In [23]:
# Show everything except the middle
df

,index,time,latitude,longitude,depth,mag,magType,rms,net,place,type,status
0,0,2020-01-28T23:20:04.732Z,61.427500,-149.908900,30.80,1.60,ml,0.6300,ak,"10km SSE of Big Lake, Alaska",earthquake,automatic
1,1,2020-01-28T23:18:37.420Z,35.057333,-118.322167,0.07,1.99,ml,0.2300,ci,"14km W of Mojave, CA",earthquake,automatic
2,2,2020-01-28T23:07:06.490Z,33.456833,-116.568667,11.22,1.66,ml,0.1800,ci,"15km SE of Anza, CA",earthquake,automatic
3,3,2020-01-28T23:02:20.720Z,33.216333,-116.361500,22.03,0.86,ml,0.2600,ci,"5km SSE of Borrego Springs, CA",earthquake,automatic
4,4,2020-01-28T22:45:01.308Z,19.075800,-80.137500,10.00,4.40,mb,0.8400,us,"105km ESE of East End, Cayman Islands",earthquake,reviewed
...,...,...,...,...,...,...,...,...,...,...,...,...
12391,12392,2019-12-29T23:53:13.606Z,-50.390300,130.765400,10.00,4.50,mb,0.9100,us,Western Indian-Antarctic Ridge,earthquake,reviewed
12392,12393,2019-12-29T23:42:46.710Z,17.899100,-66.928500,14.00,2.07,md,0.0900,pr,"8km SSW of Guanica, Puerto Rico",earthquake,reviewed
12393,12394,2019-12-29T23:35:59.160Z,17.926100,-66.839100,7.00,2.16,md,0.2600,pr,"7km SSW of Indios, Puerto Rico",earthquake,reviewed
12394,12395,2019-12-29T23:32:44.496Z,37.360700,-115.641100,11.00,0.70,ml,0.1284,nn,"42km W of Alamo, Nevada",earthquake,reviewed


In [24]:
# Generate how many quakes were on the scale of 2.5 or less
less_than_2half_mag = df[df['mag'] <= 2.5]
print(less_than_2half_mag['mag'].value_counts().sum())
print('That\'s',(9824/12396)*100,'percent of quakes were 2.5 or less.')

9824
That's 79.2513714101323 percent of quakes were 2.5 or less.


In [25]:
# Generate how many quakes are in each range
# 2.5 - 2.9
between_25_29_mag = df[(df['mag'] >= 2.5) & (df['mag'] >= 2.9)]
print('2.5 - 2.9:', between_25_29_mag['mag'].value_counts().sum())

# 3 - 3.4
between_3_34_mag = df[(df['mag'] >= 3) & (df['mag'] >= 3.4)]
print('3 - 3.4:', between_3_34_mag['mag'].value_counts().sum())

# 3.5 - 3.9
between_35_39_mag = df[(df['mag'] >= 3.5) & (df['mag'] >= 3.9)]
print('3.5 - 3.9:', between_35_39_mag['mag'].value_counts().sum())

# 4 - 4.4
between_4_44_mag = df[(df['mag'] >= 4) & (df['mag'] >= 4.4)]
print('4 - 4.4:', between_4_44_mag['mag'].value_counts().sum())

# 4.5 - 4.9
between_45_49_mag = df[(df['mag'] >= 4.5) & (df['mag'] >= 4.9)]
print('4.5 - 4.9:', between_45_49_mag['mag'].value_counts().sum())

# 5 - 5.4
between_5_54_mag = df[(df['mag'] >= 5) & (df['mag'] >= 5.4)]
print('5 - 5.4:', between_5_54_mag['mag'].value_counts().sum())

# 5.5 - 5.9
between_55_59_mag = df[(df['mag'] >= 5.5) & (df['mag'] >= 5.9)]
print('5.5 - 5.9:', between_55_59_mag['mag'].value_counts().sum())

# 6 - 6.4
between_6_64_mag = df[(df['mag'] >= 6) & (df['mag'] >= 6.4)]
print('6 - 6.4:', between_6_64_mag['mag'].value_counts().sum())

# 6.5 - 6.9
between_65_69_mag = df[(df['mag'] >= 6.5) & (df['mag'] >= 6.9)]
print('6.5 - 6.9:', between_65_69_mag['mag'].value_counts().sum())

# 7 - 7.4
between_7_74_mag = df[(df['mag'] >= 7) & (df['mag'] >= 7.4)]
print('7 - 7.4:', between_7_74_mag['mag'].value_counts().sum())

# 7.5 - 7.9
between_75_79_mag = df[(df['mag'] >= 7.5) & (df['mag'] >= 7.9)]
print('7.5 - 7.9:', between_75_79_mag['mag'].value_counts().sum())

# 8 - 8.4
between_8_84_mag = df[(df['mag'] >= 8) & (df['mag'] >= 8.4)]
print('8 - 8.4:', between_8_84_mag['mag'].value_counts().sum())

# 8.5 - 8.9
between_85_89_mag = df[(df['mag'] >= 8.5) & (df['mag'] >= 8.9)]
print('8.5 - 8.9:', between_85_89_mag['mag'].value_counts().sum())

# 9 - 9.4
between_9_94_mag = df[(df['mag'] >= 9) & (df['mag'] >= 9.4)]
print('9 - 9.4:', between_9_94_mag['mag'].value_counts().sum())

# 9.5+
between_95_plus_mag = df[(df['mag'] >= 9.5)]
print('9.5+:', between_95_plus_mag['mag'].value_counts().sum())

2.5 - 2.9: 1567
3 - 3.4: 1060
3.5 - 3.9: 857
4 - 4.4: 583
4.5 - 4.9: 197
5 - 5.4: 51
5.5 - 5.9: 16
6 - 6.4: 3
6.5 - 6.9: 1
7 - 7.4: 1
7.5 - 7.9: 0
8 - 8.4: 0
8.5 - 8.9: 0
9 - 9.4: 0
9.5+: 0


In [26]:
# Copy the main df for some remodeling
df_copy1 = df.copy()
df_copy1

,index,time,latitude,longitude,depth,mag,magType,rms,net,place,type,status
0,0,2020-01-28T23:20:04.732Z,61.427500,-149.908900,30.80,1.60,ml,0.6300,ak,"10km SSE of Big Lake, Alaska",earthquake,automatic
1,1,2020-01-28T23:18:37.420Z,35.057333,-118.322167,0.07,1.99,ml,0.2300,ci,"14km W of Mojave, CA",earthquake,automatic
2,2,2020-01-28T23:07:06.490Z,33.456833,-116.568667,11.22,1.66,ml,0.1800,ci,"15km SE of Anza, CA",earthquake,automatic
3,3,2020-01-28T23:02:20.720Z,33.216333,-116.361500,22.03,0.86,ml,0.2600,ci,"5km SSE of Borrego Springs, CA",earthquake,automatic
4,4,2020-01-28T22:45:01.308Z,19.075800,-80.137500,10.00,4.40,mb,0.8400,us,"105km ESE of East End, Cayman Islands",earthquake,reviewed
...,...,...,...,...,...,...,...,...,...,...,...,...
12391,12392,2019-12-29T23:53:13.606Z,-50.390300,130.765400,10.00,4.50,mb,0.9100,us,Western Indian-Antarctic Ridge,earthquake,reviewed
12392,12393,2019-12-29T23:42:46.710Z,17.899100,-66.928500,14.00,2.07,md,0.0900,pr,"8km SSW of Guanica, Puerto Rico",earthquake,reviewed
12393,12394,2019-12-29T23:35:59.160Z,17.926100,-66.839100,7.00,2.16,md,0.2600,pr,"7km SSW of Indios, Puerto Rico",earthquake,reviewed
12394,12395,2019-12-29T23:32:44.496Z,37.360700,-115.641100,11.00,0.70,ml,0.1284,nn,"42km W of Alamo, Nevada",earthquake,reviewed


In [27]:
df_copy1.drop(columns=['magType','status','rms','type','net'], inplace=True)
df_copy1

,index,time,latitude,longitude,depth,mag,place
0,0,2020-01-28T23:20:04.732Z,61.427500,-149.908900,30.80,1.60,"10km SSE of Big Lake, Alaska"
1,1,2020-01-28T23:18:37.420Z,35.057333,-118.322167,0.07,1.99,"14km W of Mojave, CA"
2,2,2020-01-28T23:07:06.490Z,33.456833,-116.568667,11.22,1.66,"15km SE of Anza, CA"
3,3,2020-01-28T23:02:20.720Z,33.216333,-116.361500,22.03,0.86,"5km SSE of Borrego Springs, CA"
4,4,2020-01-28T22:45:01.308Z,19.075800,-80.137500,10.00,4.40,"105km ESE of East End, Cayman Islands"
...,...,...,...,...,...,...,...
12391,12392,2019-12-29T23:53:13.606Z,-50.390300,130.765400,10.00,4.50,Western Indian-Antarctic Ridge
12392,12393,2019-12-29T23:42:46.710Z,17.899100,-66.928500,14.00,2.07,"8km SSW of Guanica, Puerto Rico"
12393,12394,2019-12-29T23:35:59.160Z,17.926100,-66.839100,7.00,2.16,"7km SSW of Indios, Puerto Rico"
12394,12395,2019-12-29T23:32:44.496Z,37.360700,-115.641100,11.00,0.70,"42km W of Alamo, Nevada"


In [28]:
place_mag_df = place_df.concat(mag_df)
place_mag_df

AttributeError: ignored